# Pytorch classifier notebook

V1 : only 1 split. First implementation  
All folds V1 : with all folds  
All folds V2 : add activation stats plot  
All folds V3 : try 2 outputs regression and classification. Activation stats plot fixed.  
All folds V4 : still 2 outputs regression and classification, but 2 variables on last layer, instead of 2 last layers.  
All folds V5 : Model from kaggle : https://www.kaggle.com/c/jane-street-market-prediction/discussion/214321

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.utils as utils

from torch.nn.modules.loss import _WeightedLoss

from torch.utils.tensorboard import SummaryWriter
import io
import PIL.Image

torch.set_printoptions(edgeitems=2)
torch.manual_seed(42)

DATASET_INPUT_FILE = 'train.csv'

#FEATURES_LIST_TOTRAIN = ['feature_'+str(i) for i in range(130)] + ['cross_41_42_43', 'cross_1_2']
FEATURES_LIST_TOTRAIN = ['feature_'+str(i) for i in range(130)]

# For custom non-overlaped folds generation
TRAIN_PERCENT = 0.70  
TEST_PERCENT = 0.30

#pd.set_option('display.max_rows', 500)

In [2]:
# Deterministic Behavior
seed = 42
#os.environ['PYTHONHASHSEED'] = str(seed)
# Torch RNG
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# Python RNG
np.random.seed(seed)
#random.seed(seed)
# CuDA Determinism
torch.backends.cudnn.deterministic = True
#torch.backends.cudnn.benchmark = False

In [58]:
#BATCH_SIZE = 50000
#BATCH_SIZE = 4096 # Gave once better results than 50000
#BATCH_SIZE = 2048

#BATCH_SIZE = 300000

#BATCH_SIZE = 500000

#BATCH_SIZE = 100000
BATCH_SIZE = 50000

#BATCH_SIZE = 8192
#BATCH_SIZE = 40960

NUM_EPOCHS = 100
#MODEL_COMMENT = "All folds, 3 layers 2000, 1000 and 1, batch size 4096, lr=1e-4, patience 5, standard scale, 0.7 dropout, activ stats, CLASSIF+REG without sigmoid on reg"
MODEL_COMMENT = "All folds, kaggle resnet, without cross features, classif, batch size 8192, weight decay 1e-5, lr=1e-4, patience 5, no standard scale"

In [4]:
def pyStandardScale(tensor, mean, std):
    return((tensor - mean) / std)

In [5]:
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

from matplotlib.colors import ListedColormap
    
# this is code slightly modified from the sklearn docs here:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py
def plot_cv_indices(cv, X, y, group, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""
    
    cmap_cv = plt.cm.coolwarm

    jet = plt.cm.get_cmap('jet', 256)
    seq = np.linspace(0, 1, 256)
    _ = np.random.shuffle(seq)   # inplace
    cmap_data = ListedColormap(jet(seq))

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=group)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(range(len(indices)), [ii + .5] * len(indices),
                   c=indices, marker='_', lw=lw, cmap=cmap_cv,
                   vmin=-.2, vmax=1.2)

    # Plot the data classes and groups at the end
    ax.scatter(range(len(X)), [ii + 1.5] * len(X),
               c=y, marker='_', lw=lw, cmap=plt.cm.Set3)

    ax.scatter(range(len(X)), [ii + 2.5] * len(X),
               c=group, marker='_', lw=lw, cmap=cmap_data)

    # Formatting
    yticklabels = list(range(n_splits)) + ['target', 'day']
    ax.set(yticks=np.arange(n_splits+2) + .5, yticklabels=yticklabels,
           xlabel='Sample index', ylabel="CV iteration",
           ylim=[n_splits+2.2, -.2], xlim=[0, len(y)])
    ax.set_title('{}'.format(type(cv).__name__), fontsize=15)
    return ax


def plot_cv_indices_custom(cv_custom, X, y, group, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""
    
    cmap_cv = plt.cm.coolwarm

    jet = plt.cm.get_cmap('jet', 256)
    seq = np.linspace(0, 1, 256)
    _ = np.random.shuffle(seq)   # inplace
    cmap_data = ListedColormap(jet(seq))

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv_custom):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(range(len(indices)), [ii + .5] * len(indices),
                   c=indices, marker='_', lw=lw, cmap=cmap_cv,
                   vmin=-.2, vmax=1.2)

    # Plot the data classes and groups at the end
    ax.scatter(range(len(X)), [ii + 1.5] * len(X),
               c=y, marker='_', lw=lw, cmap=plt.cm.Set3)

    ax.scatter(range(len(X)), [ii + 2.5] * len(X),
               c=group, marker='_', lw=lw, cmap=cmap_data)

    # Formatting
    yticklabels = list(range(n_splits)) + ['target', 'day']
    ax.set(yticks=np.arange(n_splits+2) + .5, yticklabels=yticklabels,
           xlabel='Sample index', ylabel="CV iteration",
           ylim=[n_splits+2.2, -.2], xlim=[0, len(y)])
    ax.set_title('{}'.format(type(cv).__name__), fontsize=15)
    return ax

In [6]:
# This function accounts for variable instance counts in each split by dividing utility_pi by number of instances (but this has been removed)
# It also does some copy of dataframe to prevent memory overwrite
def utility_function(df_test, df_test_predictions):
    df_test_copy = df_test.copy(deep=True)
    df_test_copy.loc[:, 'utility_pj'] = df_test_copy['weight'] * df_test_copy['resp'] * df_test_predictions
    #df_test_utility_pi = df_test_copy.groupby('date')['utility_pj'].sum() / df_test_copy.groupby('date')['utility_pj'].count()
    df_test_utility_pi = df_test_copy.groupby('date')['utility_pj'].sum()

    nb_unique_dates = df_test_utility_pi.shape[0]
    
    if (np.sqrt(df_test_utility_pi.pow(2).sum()) == 0):
        t = 0

    else:
        t = (df_test_utility_pi.sum() / np.sqrt(df_test_utility_pi.pow(2).sum())) * (np.sqrt(250 / np.abs(nb_unique_dates)))
        
    u = min(max(t, 0), 6) * df_test_utility_pi.sum()
    del df_test_copy
    
    return(u)

In [7]:
from bisect import bisect_left

def take_closest(myList, myNumber):
    """
    Assumes myList is sorted. Returns closest value to myNumber.

    If two numbers are equally close, return the smallest number.
    """
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return myList[0]
    if pos == len(myList):
        return myList[-1]
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
       return after
    else:
       return before

In [8]:
class SaveOutputActivationStats:
    def __init__(self):
        self.outputs = []
        
    def __call__(self, module, module_in, module_out):
        #self.outputs.append(module_out)
        self.outputs.append({'mean': module_out.mean().item(), 'std': module_out.std().item(),'near_zero': (module_out<=0.05).long().sum().item()/module_out.numel()})
        
    def clear(self):
        self.outputs = []

In [9]:
#fig, ax = plt.subplots(1, 1)
#
#plot_cv_indices(cv, df.loc[:, FEATURES_LIST_TOTRAIN], (df['resp'] > 0), df['date'], 
#                         ax, 5, lw=20);

In [10]:
torch.device

torch.device

In [11]:
torch.cuda.is_available()

True

In [12]:
torch.cuda.current_device()

0

In [13]:
torch.cuda.get_device_name(0)

'GeForce RTX 3090'

# Load data

In [14]:
# Load data
    
df = pd.read_csv(DATASET_INPUT_FILE)
df['resp_positive'] = ((df['resp'])>0)*1  # Target to predict

print('Data loaded')


Data loaded


# Feature engineering

In [15]:
df['cross_41_42_43'] = df['feature_41'] + df['feature_42'] + df['feature_43']

In [16]:
df['cross_1_2'] = df['feature_1'] / (df['feature_2'] + 1e-5)

# Non overlap fold generation

In [17]:
date_indexes_list = df.groupby('date')['ts_id'].first().to_list()

In [18]:
base_train_split_size = int((df.shape[0] // 5) * TRAIN_PERCENT)

In [19]:
base_test_split_size = int((df.shape[0] // 5) * TEST_PERCENT)

In [20]:
train_split_start_indexes = [take_closest(date_indexes_list, (base_train_split_size + base_test_split_size)*fold_indice) for fold_indice in range(5)]

In [21]:
test_split_start_indexes = [take_closest(date_indexes_list, (base_train_split_size + base_test_split_size)*fold_indice) for fold_indice in range(5)]

In [22]:
train_split_start_indexes

[0, 477711, 958233, 1435933, 1913985]

In [23]:
df.shape[0] - 1

2390490

In [24]:
NB_FOLDS = 5
last_index = df.shape[0] - 1

cv_table = []

for fold_indice in range(NB_FOLDS):
    fold_train_start_index = train_split_start_indexes[fold_indice]
    
    if (fold_indice == NB_FOLDS - 1):    
        nextfold_train_start_index = last_index
        
    else:
        nextfold_train_start_index = train_split_start_indexes[fold_indice + 1]
    
    fold_test_start_index = take_closest(date_indexes_list, int(TRAIN_PERCENT * (nextfold_train_start_index - fold_train_start_index) + fold_train_start_index  ))
    
    cv_table.append(fold_train_start_index)
    cv_table.append(fold_test_start_index)

In [25]:
cv_table.append(last_index)

In [26]:
cv_tuples = []

for i in range(0, NB_FOLDS*2, 2):
    cv_tuples.append([df.loc[cv_table[i]:cv_table[i+1]-1, :].index.to_list(), df.loc[cv_table[i+1]:cv_table[i+2]-1, :].index.to_list()])

In [27]:
cv_tuples_generator = iter(cv_tuples)

In [28]:
#fig, ax = plt.subplots(1, 1)

#plot_cv_indices_custom(cv_tuples_generator, df.loc[:, FEATURES_LIST_TOTRAIN], (df['resp'] > 0), df['date'], 
#                         ax, 5, lw=20); 

#cv_tuples_generator = iter(cv_tuples)

In [29]:
# Size of training set :
#train_sets_table =  [cv_tuples[i][0] for i in range(5)]
#sum([len(train_set_table) for train_set_table in train_sets_table])

In [30]:
# Our old time series split (with overlap : required 1 neural network trained per split)
# But in this script it's not needed because we're training 1 unique network, with a different fold strategy (non overlaped)
#cv = PurgedGroupTimeSeriesSplit(
#    n_splits=5,
#    max_train_group_size=180,
#    group_gap=20,
#    max_test_group_size=60
#)

In [31]:
#train_index, test_index = next(cv.split(df, (df['resp'] > 0)*1, df['date']))

In [32]:
#(df.loc[train_index, 'resp'] > 0).astype(np.byte)

In [33]:
f_mean = df.loc[:, FEATURES_LIST_TOTRAIN].mean(axis=0)

In [34]:
f_mean.shape

(130,)

In [35]:
df.fillna(f_mean, inplace=True)

In [36]:
#print('Sum of model parameters:')
#[print(p.sum()) for p in model.parameters()]

In [37]:
#writer = SummaryWriter()

#writer.add_text('test', 'test:'  + str(model).replace('\n', '<BR>'))

#writer.flush()
#writer.close()

In [38]:
#str([p.numel() for p in model.parameters()])

In [39]:
folds_list = []

for fold, (train_index, test_index) in enumerate(cv_tuples_generator):
    folds_list.append((train_index, test_index))

In [40]:
folds_list_train = [folds_list[i][0] for i in range(5)]
folds_list_train_flat = [folds_list_train_item for sublist in folds_list_train for folds_list_train_item in sublist]
folds_list_train_unique = list(set(folds_list_train_flat))

In [41]:
len(folds_list_train_unique)

1677155

In [42]:
np.sum([len(folds_list_train_item) for folds_list_train_item in folds_list_train])

1677155

In [43]:
len(folds_list_train_flat)

1677155

In [44]:
folds_list_test = [folds_list[i][1] for i in range(5)]
folds_list_test_flat = [folds_list_test_item for sublist in folds_list_test for folds_list_test_item in sublist]
folds_list_test_unique = set(folds_list_test_flat)

In [45]:
np.sum([len(folds_list_test_item) for folds_list_test_item in folds_list_test])

713335

In [46]:
len(folds_list_test_flat)

713335

In [47]:
len(folds_list_train_flat) + len(folds_list_test_flat)

2390490

In [48]:
df.loc[folds_list_test[4], FEATURES_LIST_TOTRAIN].to_numpy().shape

(141980, 130)

In [49]:
df.shape

(2390491, 141)

# Training

In [59]:
print('Training started')
patience=5
label_smoothing = 1e-2

utility_scores = [None] * 5
accuracy_scores = [None] * 5

writer = SummaryWriter()

ts_train = torch.tensor(df.loc[folds_list_train_unique, FEATURES_LIST_TOTRAIN].to_numpy(), device='cuda')
ts_train_y = torch.tensor((df.loc[folds_list_train_unique, 'resp'] > 0).astype(np.byte).to_numpy(), device='cuda')
ts_train_y_reg = torch.tensor(df.loc[folds_list_train_unique, 'resp'].to_numpy(), device='cuda')

# Normalize data
ts_train_mean = torch.mean(ts_train, axis=0)
ts_train_std = torch.std(ts_train, axis=0)
#ts_train = pyStandardScale(ts_train, ts_train_mean, ts_train_std)

train_dataset = torch.utils.data.TensorDataset(ts_train, ts_train_y, ts_train_y_reg)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True) # pin_memory : VOIR RESULTAT

ts_test = [None] * 5
ts_test_y = [None] * 5    
ts_test_y_reg = [None] * 5   
test_dataset = [None] * 5
test_loader = [None] * 5

for fold_indice in range(5):
    ts_test[fold_indice] = torch.tensor(df.loc[folds_list_test[fold_indice], FEATURES_LIST_TOTRAIN].to_numpy(), device='cuda')
    ts_test_y[fold_indice] = torch.tensor((df.loc[folds_list_test[fold_indice], 'resp'] > 0).astype(np.byte).to_numpy(), device='cuda')
    ts_test_y_reg[fold_indice] = torch.tensor(df.loc[folds_list_test[fold_indice], 'resp'].to_numpy(), device='cuda')

    # Normalize
    #ts_test[fold_indice] = pyStandardScale(ts_test[fold_indice], ts_train_mean, ts_train_std)
    
    test_dataset[fold_indice] = torch.utils.data.TensorDataset(ts_test[fold_indice], ts_test_y[fold_indice], ts_test_y_reg[fold_indice])
    test_loader[fold_indice] = torch.utils.data.DataLoader(test_dataset[fold_indice], batch_size=BATCH_SIZE)

torch.manual_seed(seed)

'''
model = nn.Sequential(
        #nn.Dropout(0.2),
        nn.Linear(len(FEATURES_LIST_TOTRAIN), 200),
        #nn.BatchNorm1d(130),
        nn.ReLU(),
        nn.Dropout(0.7),

        nn.Linear(200, 100),
        #nn.BatchNorm1d(130),
        nn.ReLU(),
        nn.Dropout(0.7),
    
        nn.Linear(100, 1),
        nn.Sigmoid(),
    ).double().to('cuda')
'''

class MLP(torch.nn.Module):
    #def __init__(self, n_feature, n_hidden): 
    def __init__(self): 
        super(MLP, self).__init__() 

        #nn.Dropout(0.2),
        self.layer1 = nn.Linear(len(FEATURES_LIST_TOTRAIN), 130)
        #self.batchnorm1 = nn.BatchNorm1d(130)
        self.act1 = nn.Tanh()
        self.dropout1 = nn.Dropout(0.3)

        self.layer2 = nn.Linear(130, 60)
        #self.batchnorm2 = nn.BatchNorm1d(130)
        self.act2 = nn.Tanh()
        self.dropout2 = nn.Dropout(0.3)

        self.layer3 = nn.Linear(60, 1)
        
        #self.layer11_classif = nn.Linear(30, 1)
        #self.act11_classif = nn.Sigmoid()  

        #self.layer11_reg = nn.Linear(30, 1)



    def forward(self, x):
        x = self.dropout1(self.act1(self.layer1(x)))
        x = self.dropout2(self.act2(self.layer2(x)))
        x = self.layer3(x)

        #x_out_classif = nn.Sigmoid()(x[:, 0]).unsqueeze(-1)
        #x_out_reg = x[:, 1].unsqueeze(-1)

        #x_out_classif = nn.Sigmoid()(x[:, 0])
        #x_out_reg = x[:, 1]
        x_out_reg = x

        #return x_out_classif, x_out_reg
        return x_out_reg

class Model_Resnet(nn.Module):
    def __init__(self):
        super(Model_Resnet, self).__init__()
        self.batch_norm0 = nn.BatchNorm1d(len(FEATURES_LIST_TOTRAIN))
        self.dropout0 = nn.Dropout(0.2)

        dropout_rate = 0.2
        hidden_size = 256
        self.dense1 = nn.Linear(len(FEATURES_LIST_TOTRAIN), hidden_size)
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.dense2 = nn.Linear(hidden_size+len(FEATURES_LIST_TOTRAIN), hidden_size)
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.dense3 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rate)

        self.dense4 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(dropout_rate)

        #self.dense5 = nn.Linear(hidden_size+hidden_size, len(target_cols))
        self.dense5 = nn.Linear(hidden_size+hidden_size, 1)

        self.act1 = nn.LeakyReLU()
        self.act2 = nn.LeakyReLU()
        self.act3 = nn.LeakyReLU()
        self.act4 = nn.LeakyReLU()
        self.actfinal = nn.Sigmoid()
        
        ##self.Relu = nn.ReLU(inplace=True)
        ##self.PReLU = nn.PReLU()
        ##self.LeakyReLU = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        # self.GeLU = nn.GELU()
        ##self.RReLU = nn.RReLU()

    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x1 = self.dense1(x)
        x1 = self.batch_norm1(x1)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x1 = self.act1(x1)
        x1 = self.dropout1(x1)

        x = torch.cat([x, x1], 1)

        x2 = self.dense2(x)
        x2 = self.batch_norm2(x2)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x2 = self.act2(x2)
        x2 = self.dropout2(x2)

        x = torch.cat([x1, x2], 1)

        x3 = self.dense3(x)
        x3 = self.batch_norm3(x3)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x3 = self.act3(x3)
        x3 = self.dropout3(x3)

        x = torch.cat([x2, x3], 1)

        x4 = self.dense4(x)
        x4 = self.batch_norm4(x4)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x4 = self.act4(x4)
        x4 = self.dropout4(x4)

        x = torch.cat([x3, x4], 1)

        x = self.actfinal(self.dense5(x))

        return x

    
#model = MLP().double().to('cuda')
model = Model_Resnet().double().to('cuda')
    
print('Model created')

#print('Number of model parameters :')
#numel_list = [p.numel() for p in model.parameters()]
#sum(numel_list), numel_list


class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

#loss_fn = nn.BCELoss().to('cuda')
loss_fn = SmoothBCEwLogits(smoothing=label_smoothing)


#loss_fn_reg = nn.MSELoss().to('cuda')

#def MSE_SignedLoss(output, target):
    #return((nn.LeakyReLU()(-(output*target)) / torch.abs(output*target)) * loss_fn_reg(output, target))

def MSE_SignedLoss(output, target):
    return(
        torch.sqrt(
            torch.mean(
                (nn.LeakyReLU()(-(output*target)) / torch.abs(output*target)) * torch.pow(output - target, 2)
        )
    )
    )



optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5) 
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-2) 

model.eval()
#start_accuracy = accuracy_score(ts_test_y.cpu().numpy(), (model(ts_test).squeeze() > 0.5).cpu().numpy())
#start_utility_score = utility_function(df.loc[test_index], (model(ts_test).squeeze() > 0.5).cpu().numpy())
#print('Start Validation Accuracy: {:.4f}'.format(start_accuracy))
#print('Start Validation Utility: {:.4f}'.format(start_utility_score))

### Call back to save activation stats (mean, std dev and near 0 values after activation functions)

Val_Loss = 0
N_Samples = 0

the_last_loss = 100
the_last_utility_score = 0
the_last_accuracy = 0
trigger_times=0
early_stopping_met = False

for epoch in range(NUM_EPOCHS): 
    running_loss = 0.0        
    
    # Setting hook for activation layers stats
    hook_handles = []
    save_output_activation_stats = []

    for layer in model.modules():
        if ('activation' in str(type(layer))):
            save_output_activation_stats_1layer = SaveOutputActivationStats()
            handle = layer.register_forward_hook(save_output_activation_stats_1layer)
            save_output_activation_stats.append(save_output_activation_stats_1layer)
            hook_handles.append(handle)
    
    model.train()

    for batch in train_loader:
        #inputs, labels = batch[0], batch[1]
        inputs, labels, labels_reg = batch[0].to('cuda'), batch[1].to('cuda'), batch[2].to('cuda')
        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            #outputs_classif, outputs_reg = model(inputs)
            #outputs_reg = model(inputs)
            outputs_classif = model(inputs)
            loss_classif = loss_fn(outputs_classif, labels.unsqueeze(-1).double())
            
            #loss_classif = 0
            #loss_reg = MSE_SignedLoss(outputs_reg, labels_reg.unsqueeze(-1).double()) * torch.tensor(864.625) # Coefficient to balance reg loss which is much smaller
            #loss_reg = MSE_SignedLoss(outputs_reg, labels_reg.unsqueeze(-1).double()) # Coefficient to balance reg loss which is much smaller
            #loss_reg = loss_fn_reg(outputs_reg.unsqueeze(-1), labels_reg.unsqueeze(-1).double()) # Coefficient to balance reg loss which is much smaller
            
            #loss = loss_classif + loss_reg
            #loss = loss_reg
            loss = loss_classif
            
            loss.backward()
            optimizer.step()

    # update local train loss
        running_loss += loss.item() * inputs.size(0)

    # update global train loss
    epoch_loss = running_loss / len(train_loader.dataset)
    print('Epoch({}) - Training Loss: {:.4f}'.format(epoch, epoch_loss))

    writer.add_scalar(f"Global train/loss", epoch_loss, epoch)

    # Write activation stats graphs
    for layer_number,save_output_activation_stats_layer in enumerate(save_output_activation_stats):
        df_stats_layer = pd.DataFrame(save_output_activation_stats_layer.outputs)

        fig, ax = plt.subplots(1, 3, figsize=(25, 4))

        ax[0].set_title(f'Layer {layer_number} : Mean activation value', fontsize=16)
        ax[0].set_xlabel('Batch instances')
        ax[0].set_ylabel('Mean')
        ax[0].plot(range(df_stats_layer.shape[0]), df_stats_layer['mean'])

        ax[1].set_title(f'Layer {layer_number} : Std deviation activation value', fontsize=16)
        ax[1].set_xlabel('Batch instances')
        ax[1].set_ylabel('Standard deviation')
        ax[1].plot(range(df_stats_layer.shape[0]), df_stats_layer['std'])

        ax[2].set_title(f'Layer {layer_number} : Percentage of activation values near zero', fontsize=16)
        ax[2].set_xlabel('Batch instances')
        ax[2].set_ylabel('Percentage')
        ax[2].plot(range(df_stats_layer.shape[0]), df_stats_layer['near_zero']);
        
        plot_buf = io.BytesIO()
        plt.savefig(plot_buf, format='jpeg')
        plt.close()
        
        plot_buf.seek(0)
        image = PIL.Image.open(plot_buf)
        image = transforms.ToTensor()(image)
        writer.add_image("Train activation stats/Activation stats layer " + str(layer_number), image, epoch)
    
    # Validation 
    model.eval()

    vrunning_loss = [None] * 5
    vrunning_loss_classif = [None] * 5
    vrunning_loss_reg = [None] * 5
    num_samples = [None] * 5
    vepoch_loss_folds = [None] * 5
    vepoch_loss_classif_folds = [None] * 5
    vepoch_loss_reg_folds = [None] * 5
    vepoch_accuracy_folds = [None] * 5
    vepoch_utility_score_folds = [None] * 5
    
    for fold_indice in range(5):    
        vrunning_loss[fold_indice] = 0.0
        vrunning_loss_classif[fold_indice] = 0.0
        vrunning_loss_reg[fold_indice] = 0.0
        num_samples[fold_indice] = 0

        for batch in test_loader[fold_indice]:
            inputs, labels, labels_reg = batch[0].to('cuda'), batch[1].to('cuda'), batch[2].to('cuda')

            optimizer.zero_grad()
            with torch.no_grad():
                #outputs_classif, outputs_reg = model(inputs)
                outputs_classif = model(inputs)
                loss_classif = loss_fn(outputs_classif, labels.unsqueeze(-1).double())

                #loss_reg = loss_fn_reg(outputs_reg, torch.sigmoid(labels_reg.unsqueeze(-1).double()))
                #loss_reg = loss_fn_reg(outputs_reg.unsqueeze(-1), labels_reg.unsqueeze(-1).double()) # Coefficient to balance reg loss which is much smaller
                #loss_reg = MSE_SignedLoss(outputs_reg, labels_reg.unsqueeze(-1).double()) * torch.tensor(864.625) # Coefficient to balance reg loss which is much smaller
                #loss_reg = MSE_SignedLoss(outputs_reg, labels_reg.unsqueeze(-1).double()) # Coefficient to balance reg loss which is much smaller
                
                #loss = loss_classif + loss_reg 
                loss = loss_classif 
                #loss = loss_reg 
                
            vrunning_loss[fold_indice] += loss.item() * inputs.size(0)
            num_samples[fold_indice] += labels.size(0)
            
            vrunning_loss_classif[fold_indice] += loss_classif.item() * inputs.size(0)
            #running_loss_reg[fold_indice] += loss_reg.item() * inputs.size(0)
            
            vepoch_loss_folds[fold_indice] = vrunning_loss[fold_indice] / num_samples[fold_indice]
            vepoch_loss_classif_folds[fold_indice] = vrunning_loss_classif[fold_indice] / num_samples[fold_indice]
            #epoch_loss_reg_folds[fold_indice] = vrunning_loss_reg[fold_indice] / num_samples[fold_indice]

        print('Epoch({}) - Fold {} - Validation Loss : {:.4f}'.format(epoch, fold_indice, vepoch_loss_folds[fold_indice]))
        #print('Epoch({}) - Fold {} -> Validation Loss Classif : {:.4f}'.format(epoch, fold_indice, vepoch_loss_classif_folds[fold_indice]))
        #print('Epoch({}) - Fold {} -> Validation Loss Reg : {:.4f}'.format(epoch, fold_indice, vepoch_loss_reg_folds[fold_indice]))

        model.eval()
        with torch.no_grad():
            vepoch_accuracy_folds[fold_indice] = accuracy_score(ts_test_y[fold_indice].cpu().numpy(), (model(ts_test[fold_indice]).squeeze() > 0.5).cpu().numpy())  # model...[0] is the classification output of model
            vepoch_utility_score_folds[fold_indice] = utility_function(df.loc[folds_list_test[fold_indice]], (model(ts_test[fold_indice]).squeeze() > 0.5).cpu().numpy())
        print('Epoch({}) - Fold {} - Validation Accuracy : {:.4f}'.format(epoch, fold_indice, vepoch_accuracy_folds[fold_indice]))
        print('Epoch({}) - Fold {} - Validation Utility score : {:.4f}'.format(epoch, fold_indice, vepoch_utility_score_folds[fold_indice]))
        
            
    # update epoch loss
    vepoch_loss = sum(vepoch_loss_folds) / len(vepoch_loss_folds)
    vepoch_accuracy = sum(vepoch_accuracy_folds) / len(vepoch_accuracy_folds)
    vepoch_utility_score = sum(vepoch_utility_score_folds) #/ len(vepoch_utility_score_folds)
    print('Epoch({}) - GLOBAL - Validation Loss: {:.4f}'.format(epoch, vepoch_loss))
    print('Epoch({}) - GLOBAL - Validation Accuracy: {:.4f}'.format(epoch, vepoch_accuracy))
    print('Epoch({}) - GLOBAL - Validation Utility score: {:.4f}'.format(epoch, vepoch_utility_score))

    #print(f'Sum of model parameters ({epoch}):')
    #[print(p.sum()) for p in model.parameters()]

    writer.add_scalar("Global valid/Loss", vepoch_loss, epoch)
    writer.add_scalar("Global valid/Accuracy", vepoch_accuracy, epoch)
    writer.add_scalar("Global valid/Utility", vepoch_utility_score, epoch)

    for fold_indice in range(5):
        writer.add_scalar("Fold valid Loss/Loss fold "+str(fold_indice), vepoch_loss_folds[fold_indice], epoch)
        writer.add_scalar("Fold valid Accuracy/Accuracy fold "+str(fold_indice), vepoch_accuracy_folds[fold_indice], epoch)
        writer.add_scalar("Fold valid Utility/Utility fold "+str(fold_indice), vepoch_utility_score_folds[fold_indice], epoch)
        
    
    writer.flush()

    #if (epoch == 7):
        #print('EPOCH 7 ATTAINED')
        #break
    
    # Check if Early Stopping
    #if vepoch_loss > the_last_loss:
    #if (vepoch_utility_score < the_last_utility_score) and (vepoch_loss > the_last_loss) and (vepoch_accuracy < the_last_accuracy):
    if (vepoch_loss > the_last_loss):
        trigger_times += 1

        print(f'Intermediate early stopping : vepoch_loss = {vepoch_loss:.4f}, the_last_loss={the_last_loss:.4f}')
        #print(f'Intermediate early stopping : vepoch_accuracy = {vepoch_accuracy:.4f}, the_last_utility_score={the_last_accuracy:.4f}')
        #print(f'Intermediate early stopping : vepoch_utility_score = {vepoch_utility_score:.4f}, the_last_utility_score={the_last_utility_score:.4f}')

        if trigger_times >= patience:
            print('Meet Early stopping!')
            early_stopping_met = True
            ##torch.save(model.state_dict(), f'model_{fold}.pt')
            break
    else:
        trigger_times = 0
        the_last_loss = vepoch_loss
        the_last_utility_score = vepoch_utility_score
        the_last_accuracy = vepoch_accuracy
        
        the_last_utility_score_folds = vepoch_utility_score_folds
        the_last_accuracy_folds = vepoch_accuracy_folds
        
        the_best_epoch = epoch

        # Save model for the best version so far
        print(f'Saving model corresponding to last_utility_score == {the_last_utility_score}')
        torch.save(model.state_dict(), f'model_NN_allfolds_V1.pt')

    print('\n')

if (early_stopping_met == False):
    print("Didn't meet early stopping : saving final model")
    # Save model if don't meet early stopping
    torch.save(model.state_dict(), f'model_NN_allfolds_V1.pt')

#utility_scores.append(the_last_utility_score)
#accuracy_scores.append(the_last_accuracy)
writer.add_text(f"Global valid/Utility", f"Best utility: {the_last_utility_score}", the_best_epoch)
        
scores_results = {'utility_score': the_last_utility_score, 'utility_scores': the_last_utility_score_folds, 'utility_score_std': np.std(the_last_utility_score_folds), 'accuracy_scores': the_last_accuracy_folds}

writer.add_text('Final utility score', str(scores_results))
writer.add_text('Batch size', str(BATCH_SIZE))
writer.add_text('Patience', str(patience))
writer.add_text('Number of epochs', str(NUM_EPOCHS))
writer.add_text('Number of parameters per layer', str([p.numel() for p in model.parameters()]))
writer.add_text('Model architecture', str(model).replace('\n', '<BR>'))
writer.add_text('Comment', MODEL_COMMENT)

writer.close()

print('Training summary:')
print(scores_results)

Training started
Model created
Epoch(0) - Training Loss: 0.7076
Epoch(0) - Fold 0 - Validation Loss : 0.7062
Epoch(0) - Fold 0 - Validation Accuracy : 0.5027
Epoch(0) - Fold 0 - Validation Utility score : 250.5495
Epoch(0) - Fold 1 - Validation Loss : 0.7026
Epoch(0) - Fold 1 - Validation Accuracy : 0.4971
Epoch(0) - Fold 1 - Validation Utility score : 33.6540
Epoch(0) - Fold 2 - Validation Loss : 0.7057
Epoch(0) - Fold 2 - Validation Accuracy : 0.4985
Epoch(0) - Fold 2 - Validation Utility score : -0.0000
Epoch(0) - Fold 3 - Validation Loss : 0.7065
Epoch(0) - Fold 3 - Validation Accuracy : 0.5008
Epoch(0) - Fold 3 - Validation Utility score : -0.0000
Epoch(0) - Fold 4 - Validation Loss : 0.7044
Epoch(0) - Fold 4 - Validation Accuracy : 0.5048
Epoch(0) - Fold 4 - Validation Utility score : 237.4072
Epoch(0) - GLOBAL - Validation Loss: 0.7051
Epoch(0) - GLOBAL - Validation Accuracy: 0.5008
Epoch(0) - GLOBAL - Validation Utility score: 521.6107
Saving model corresponding to last_utility

In [51]:
vepoch_loss_reg_folds[fold_indice]

In [52]:
for layer in model.modules():
    if ('activation' in str(type(layer))):
        print(layer)

LeakyReLU(negative_slope=0.01)
LeakyReLU(negative_slope=0.01)
LeakyReLU(negative_slope=0.01)
LeakyReLU(negative_slope=0.01)
Sigmoid()


In [53]:
save_output_activation_stats_layer.outputs

[{'mean': 0.06618570669301244,
  'std': 0.12045511479785051,
  'near_zero': 0.68656},
 {'mean': 0.06547741800634416,
  'std': 0.11799794368985213,
  'near_zero': 0.68742},
 {'mean': 0.06657497575316271,
  'std': 0.12137901910311018,
  'near_zero': 0.68508},
 {'mean': 0.06731119555120808,
  'std': 0.12206110280806262,
  'near_zero': 0.68542},
 {'mean': 0.06897175735430384,
  'std': 0.1263957021922444,
  'near_zero': 0.68514},
 {'mean': 0.07181969551699932,
  'std': 0.13363219690358952,
  'near_zero': 0.68352},
 {'mean': 0.07457552371267859,
  'std': 0.13899772875287916,
  'near_zero': 0.67952},
 {'mean': 0.07537183304904381,
  'std': 0.13867310007568426,
  'near_zero': 0.67746},
 {'mean': 0.07572386272832687,
  'std': 0.13712492232879478,
  'near_zero': 0.67382},
 {'mean': 0.07535796347673133,
  'std': 0.13536153398535805,
  'near_zero': 0.6764},
 {'mean': 0.07403449311129486,
  'std': 0.13283270266522146,
  'near_zero': 0.6762},
 {'mean': 0.07038238025389097,
  'std': 0.125422125944643

In [54]:
 df_stats_layer

,mean,std,near_zero
0,0.066186,0.120455,0.686560
1,0.065477,0.117998,0.687420
2,0.066575,0.121379,0.685080
3,0.067311,0.122061,0.685420
4,0.068972,0.126396,0.685140
5,0.071820,0.133632,0.683520
6,0.074576,0.138998,0.679520
7,0.075372,0.138673,0.677460
8,0.075724,0.137125,0.673820
9,0.075358,0.135362,0.676400


In [52]:
ts_test[fold_indice]

tensor([[ 1.0000, -2.1321,  ...,  0.5874,  4.3042],
        [-1.0000,  0.6492,  ...,  1.5986,  0.5584],
        ...,
        [-1.0000,  0.2672,  ..., -1.7266,  1.3048],
        [-1.0000,  0.4237,  ..., -1.2025, -1.4359]], device='cuda:0',
       dtype=torch.float64)

# Load model

In [ ]:
'''
model_load = nn.Sequential(
        #nn.Dropout(0.2),
        nn.Linear(len(FEATURES_LIST_TOTRAIN), 130),
        #nn.BatchNorm1d(130),
        nn.ReLU(),
        nn.Dropout(0.3),

        nn.Linear(130, 130),
        #nn.BatchNorm1d(130),
        nn.ReLU(),
        nn.Dropout(0.3),    

        nn.Linear(130, 130),
        #nn.BatchNorm1d(130),
        nn.ReLU(),
        nn.Dropout(0.3),    

        nn.Linear(130, 130),
        #nn.BatchNorm1d(130),
        nn.ReLU(),
        nn.Dropout(0.3),    

        nn.Linear(130, 130),
        #nn.BatchNorm1d(130),
        nn.ReLU(),
        nn.Dropout(0.3),    

        nn.Linear(130, 130),
        #nn.BatchNorm1d(130),
        nn.ReLU(),
        nn.Dropout(0.3),    
    
        nn.Linear(130, 130),
        #nn.BatchNorm1d(130),
        nn.ReLU(),
        nn.Dropout(0.3),    
    
        nn.Linear(130, 130),
        #nn.BatchNorm1d(130),
        nn.ReLU(),
        nn.Dropout(0.3),    

        nn.Linear(130, 60),
        #nn.BatchNorm1d(130),
        nn.ReLU(),
        nn.Dropout(0.3),    
    
        nn.Linear(60, 30),
        #nn.BatchNorm1d(130),
        nn.ReLU(),
        nn.Dropout(0.3),    
       
        nn.Linear(30, 1),
        nn.Sigmoid(),
    ).double().to('cuda')
    
model_load.load_state_dict(torch.load(f'model_NN_allfolds_V1.pt',map_location=torch.device('cuda')))
'''

#model_load.eval()
#print(accuracy_score(ts_test_y.cpu().numpy(), (model_load(ts_test).squeeze() > 0.5).cpu().numpy()))
#
#model_load.eval()
#print(utility_function(df.loc[test_index], (model_load(ts_test).squeeze() > 0.5).cpu().numpy()))


# Data for model inference

## First, fill NA with :

In [ ]:
f_mean

## Then, normalize with :

In [ ]:
ts_train_mean

In [ ]:
ts_train_std

In [ ]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
        
    def __call__(self, module, module_in, module_out):
        self.outputs.append(module_out)
        print(type(module_out))
        
    def clear(self):
        self.outputs = []

In [ ]:
model2 =  nn.Sequential(
        nn.Linear(4, 2),
        nn.ReLU(),

        nn.Linear(2, 2),
        nn.ReLU(),


    
        nn.Linear(2, 1),
        nn.Sigmoid(),

)

model2.eval()

In [ ]:
for i, param_object in enumerate(model2.parameters()):
    print(f'Object {i}')
    print(param_object.shape)
    print(param_object.data)

In [ ]:
'activation' in str(type(model2[1]))

In [ ]:
isinstance(model2[1], torch.nn.ReLU)

In [ ]:
save_output = SaveOutput()

hook_handles = []

for layer in model2.modules():
    if ('activation' in str(type(layer))):
        handle = layer.register_forward_hook(save_output)
        hook_handles.append(handle)

In [ ]:
hook_handles

In [ ]:
#with torch.no_grad():
print(model2(torch.tensor([1, 1, 1, 2], dtype=torch.float)))

In [ ]:
save_output.outputs

In [ ]:
activation.append([0.11, 01.11])

In [ ]:
# Pour récupérer les gradients : https://discuss.pytorch.org/t/how-to-print-the-computed-gradient-values-for-a-network/34179
print(model2[0].weight.grad)

In [ ]:
model